In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [2]:
  df = fetch_datasets()['letter_img']
  X, y = df.data, df.target
  le = LabelEncoder()
  y = le.fit_transform(y)
  G = len(np.unique(y))
  print(np.shape(X), G, sum(y==0)*.6*.5, sum(y==1)*.6*.5)
  X[:3]

(20000, 16) 2 5779.8 220.2


array([[ 2.,  8.,  3.,  5.,  1.,  8., 13.,  0.,  6.,  6., 10.,  8.,  0.,
         8.,  0.,  8.],
       [ 5., 12.,  3.,  7.,  2., 10.,  5.,  5.,  4., 13.,  3.,  9.,  2.,
         8.,  4., 10.],
       [ 4., 11.,  6.,  8.,  6., 10.,  6.,  2.,  6., 10.,  3.,  7.,  3.,
         7.,  3.,  9.]])

In [3]:
missing_rates = np.array([0.0, 0.1, 0.2, 0.3, 0.4])
res_all = []
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  n = np.array([-1000,800])
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [4]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [5]:
leftindex = pd.MultiIndex.from_product([["0%", "10%", "20%", "30%", "40%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM         0.911$\pm$0.009  0.997$\pm$0.001   
                         LR          0.958$\pm$0.003  0.981$\pm$0.002   
                         DCT         0.807$\pm$0.056  0.993$\pm$0.001   
             sensitivity SVM         0.877$\pm$0.014  0.997$\pm$0.001   
                         LR           0.95$\pm$0.004   0.98$\pm$0.002   
                         DCT         0.723$\pm$0.074  0.993$\pm$0.001   
10%          F-1 score   SVM         0.801$\pm$0.029  0.996$\pm$0.001   
                         LR           0.94$\pm$0.008  0.979$\pm$0.001   
                         DCT          0.656$\pm$0.07  0.991$\pm$0.001   
             sensitivity SVM         0.712$\pm$0.041  0.996$\pm$0.001   
                         LR          0.924$\pm$0.013  0.979$\pm$0.001   
                         DCT         0.532$\pm$0.079  0.991$\pm$0.001   
20%          F-1 score   SVM         0.661$\pm$0.022  0.995$\pm$0.001   
                         LR          0.928$\pm$0.008  0.982$\pm$0.001   
                         DCT         0.608$\pm$0.061  0.988$\pm$0.002   
             sensitivity SVM         0.533$\pm$0.026  0.995$\pm$0.001   
                         LR          0.903$\pm$0.012  0.981$\pm$0.001   
                         DCT         0.478$\pm$0.066  0.988$\pm$0.002   
30%          F-1 score   SVM         0.426$\pm$0.036   0.99$\pm$0.002   
                         LR           0.89$\pm$0.014  0.979$\pm$0.002   
                         DCT         0.605$\pm$0.038  0.985$\pm$0.002   
             sensitivity SVM         0.307$\pm$0.028   0.99$\pm$0.002   
                         LR          0.845$\pm$0.021  0.979$\pm$0.002   
                         DCT         0.473$\pm$0.039  0.985$\pm$0.002   
40%          F-1 score   SVM          0.32$\pm$0.022  0.988$\pm$0.002   
                         LR          0.872$\pm$0.013  0.977$\pm$0.003   
                         DCT         0.537$\pm$0.072  0.978$\pm$0.003   
             sensitivity SVM         0.227$\pm$0.016  0.988$\pm$0.002   
                         LR           0.817$\pm$0.02  0.977$\pm$0.003   
                         DCT         0.409$\pm$0.067  0.978$\pm$0.003   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM           0.997$\pm$0.0    0.997$\pm$0.0   
                         LR           0.98$\pm$0.001  0.977$\pm$0.001   
                         DCT         0.993$\pm$0.001  0.993$\pm$0.001   
             sensitivity SVM           0.997$\pm$0.0    0.997$\pm$0.0   
                         LR           0.98$\pm$0.001  0.976$\pm$0.002   
                         DCT         0.993$\pm$0.001  0.993$\pm$0.001   
10%          F-1 score   SVM         0.996$\pm$0.001  0.995$\pm$0.001   
                         LR          0.979$\pm$0.001  0.977$\pm$0.001   
                         DCT         0.989$\pm$0.001  0.991$\pm$0.001   
             sensitivity SVM         0.996$\pm$0.001  0.995$\pm$0.001   
                         LR          0.978$\pm$0.001  0.976$\pm$0.002   
                         DCT         0.989$\pm$0.001  0.991$\pm$0.001   
20%          F-1 score   SVM         0.994$\pm$0.001  0.994$\pm$0.001   
                         LR           0.98$\pm$0.001  0.979$\pm$0.001   
                         DCT         0.988$\pm$0.002  0.989$\pm$0.001   
             sensitivity SVM         0.994$\pm$0.001  0.994$\pm$0.001   
                         LR          0.979$\pm$0.001  0.978$\pm$0.001   
                         DCT         0.988$\pm$0.002  0.989$\pm$0.001   
30%          F-1 score   SVM         0.987$\pm$0.002  0.991$\pm$0.001   
                         LR          0.978$\pm$0.001  0.979$\pm$0.001   
                         DCT         0.985$\pm$0.003  0.986$\pm$0.002   
             sensitivity SVM

In [6]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_letter-low.csv", index = False)

In [7]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
0\% & F-1 score & SVM &  0.911\$\textbackslash pm\$0.009 &  0.997\$\textbackslash pm\$0.001 &    0.997\$\textbackslash pm\$0.0 &    0.997\$\textbackslash pm\$0.0 &  0.991\$\textbackslash pm\$0.001 &  0.995\$\textbackslash pm\$0.001 \\
    &             & LR &  0.958\$\textbackslash pm\$0.003 &  0.981\$\textbackslash pm\$0.002 &   0.98\$\textbackslash pm\$0.001 &  0.977\$\textbackslash pm\$0.001 &  0.977\$\textbackslash pm\$0.001 &  0.981\$\textbackslash pm\$0.001 \\
    &             & DCT &  0.807\$\textbackslash pm\$0.056 &  0.993\$\textbackslash pm\$0.001 &  0.993\$\textbackslash pm\$0.001 &  0.993\$\textbackslash pm\$0.001 &  0.976\$\textbackslash pm